Hellan Herrmann Johnson Method
====

In [ ]:
from ngsolve import *
from netgen.geom2d import unit_square
import netgen.gui
%gui tk

Define product finite element space for $H(\text{div div}) \times H^1$

In [ ]:
mesh = Mesh (unit_square.GenerateMesh(maxh=0.1))
Draw (mesh)

order = 0
Sigma = HDivDiv(mesh, order=order,discontinuous=True)
Hyb = HDiv(mesh, order=order, orderinner=0, dirichlet="top|bottom|left|right")
W = H1(mesh, order=order+1, dirichlet="top|bottom|left|right")
X = FESpace( [Sigma,W,Hyb] )

print ("ndof-Sigma:", Sigma.ndof, ", ndof-Hyb:", Hyb.ndof, ", ndof-W:", W.ndof, ", ndof-X:", X.ndof)

$$A(\sigma,\alpha,u) = -\frac{1}{2}a(\sigma,\sigma) + b(\sigma,\alpha,u)$$
with
\begin{align*}
a(\sigma,\tau) & = \int_\Omega \sigma: \tau\,dx \\
b(\sigma,\alpha, v) & = \sum_T \int_T \text{div} \sigma \cdot \nabla v\,dx - \int_{\partial T} \sigma_n\cdot \nabla_t v +\alpha_n\sigma_{nn}\,ds
\end{align*}

In [ ]:
sigma, u = X.TrialFunction()

n = specialcf.normal(2)
def tang(u): return u-(u*n)*n

a = BilinearForm(X, symmetric=True)
a += SymbolicEnergy (-InnerProduct (sigma, sigma) + div(sigma)*grad(u)) 
a += SymbolicEnergy (-(sigma*n) * tang(grad(u))+(alpha*n)*(sigma*n)*n, element_boundary=True)
a += SymbolicEnergy (-500 * u)
                    
u = GridFunction(X)
r = u.vec.CreateVector()

a.Apply(u.vec, r)
a.AssembleLinearization(u.vec)

u.vec.data -= a.mat.Inverse(X.FreeDofs()) * r.vec

Draw (u.components[0], mesh, name="sigma")
Draw (u.components[1], mesh, name="disp")